<a href="https://colab.research.google.com/github/marcoandre1/preparation-TP1/blob/main/Copy_of_Pr%C3%A9paration_TP1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 0️⃣ Mise en contexte

Dans le TP1, vous devez acquérir et tester un modèle préentraîné.

Le modèle est _BARThez_ : un modèle de génération de texte **français**. Appropiré pour le _fine-tuning_ de tâches de traductions et de compréhension de textes.

Vous serez amené à _fine-tune_ le modèle avec des données collectionnées par une entreprise afin de spécifier le modèle davantage. Le but est de faire de _BARThez_ un **classifieur de tickets d'assistance TI** via _fine-tuning_.

# 1️⃣ Compréhension du modèle

## _BARThez_
_BARThez_ est un modèle basé sur _BART_ de _FAIR_ (Face AI Research). _BART_ est un modèle qui génère du texte à partir de texte. C'est un modèle _séquence-à-séquence_. Son but **n'est que de reconstruire la séquence intrant en apprenant les relations entre les mots dans un texte**.

La distiction principale entre _BARThez_ et _BART_ est que _BARThez_ a été entraîné sur 66 Go de textes francophones. Il n'a pas été entraîné pour bien réaliser une tâche spécifique ; il est prêt à se faire entraîner davantage pour une tâche. En d'autres mots, le modèle a déjà « appris » les relations entre les mots français. La partie longue et coûteuse est faite — il ne reste qu'à peaufiner.

Le modèle est implémenté avec le framework _PyTorch_. Celui-ci est un des modules les plus populaires pour le développement de réseau de neurones.

[Lire le papier publié pour _BARThez_](https://arxiv.org/abs/2010.12321)

## _Tokenizer_

Le modèle s'attend à recevoir une _séquence_ comme intrant. Il faut une logique qui exprime les éléments (_token_) d'une séquence.

Par exemple, la phrase « Je suis au campus de l'Université de Sherbrooke » peut être divisée de plusieurs manières. Elle peut être divisée par caractères, par mots, ou par règles spéciales. Par exemple, est-ce que «'l'Université » est 1 ou 2 éléments ?

Un _tokenizer_ est un objet contenant la logique pour générer une séquence d'éléments pour un modèle. Le _tokenizer_ de _BARThez_ est différent de celui de pour _BART_ : il est conçu pour la langue française.

Ces _token_ seront après envoyé au modèle. Les modèles comme _BART_ s'occupe de transformer les tokens textuels en vecteurs de nombres pour ensuite l'envoyer au modèle. Comme vu en classe, la couche d'intrants d'un réseau prend un vecteur de nombres.

La transformations de _token_ à vecteur se fait par un _embedding_. Ce mécanisme est en soi obtenu via un entraînement IA. Ce sujet est à l'extérieur de la portée du cours.

## L'architecture _transformer_

[Voir image de l'architecture de base d'un _transformer_](https://machinelearningmastery.com/wp-content/uploads/2021/08/attention_research_1.png)

En 2017, le papier scientifique _Attention Is All You Need_" a changer le monde de l'IA. Il a proposé le modèle _transformer_ pour remplacer les RNN et CNN dans les tâches des séquences de données en intrants/extrants.

Un _transformer_ a un encodeur et un décodeur. L'encodeur _BERT_ (Bidirectional Encoder Representations from Transformers) et le décodeur _GPT_ (Generative Pre-trained Transformer).

Pendant le préentraînement, l'encodeur apprend à créer une bonne représentation vectorielle de chaque _token_. Il trouve les relations statistiques entre les mots pour former des concepts. Le décodeur génère le texte le plus approprié selon les _token_ encodés. Le décodeur est préentraîné en même temps que l'encodeur.

Comme _BART_, l'architecture de _BERThez_ est un _transformer_. Cette architecture est maintenant le standard dans les tâches de traitement de langage naturel (_NLP_).

Un _transformer_ est une architecture de modèle IA sophistiquée. Celui de _BARThez_ consiste d'environ 230 sous-réseaux de neurones. Ces réseaux sont organisés en modules.

_BARThez_ a 165 millions de paramètres. Il est question de _deep learning_.

**Pour le _fine-tuning_, nous voulons modifier _BARThez_ pour aller d'un modèle séquence-à-séquence vers un modèle de classification.** Nous pourrons mettre de côté le décodeur et exploiter l'encodeur déjà optimisé.


# 2️⃣ Acquisition du modèle préentraîné

_HuggingFace_ a développé un API _Python_ pour obtenir des modèles disponibles sur leur site.

Le module `transformers` nous permet de facilement aquérir le modèle.

En premier lieu, naviguez à la page d'accueuil du modèle sur _HuggingFace_ : https://huggingface.co/moussaKam/barthez.

Souvent, le site a un bouton pour montrer le code nécessaire pour charger le modèle avec le module `transformers`.

![](https://i.ibb.co/GQ2ckHr8/transformer.png)

Le code suivant fait l'importation des sous-modules `AutoTokenizer` et `AutoModelForSeq2SeqLM` de `transformers`. Il est souvent préférable d'importer que le nécessaire afin d'éviter d'importer le tout.

Le code créé ensuite une variable `tokenizer` et `model`.

Le _tokenizer_ est celui pour _BARThez_ et le modèle est instancié

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
model = AutoModelForSeq2SeqLM.from_pretrained("moussaKam/barthez")

Testez le _tokenizer_ :

In [ ]:
prompt = "Je suis au campus de l'Université de Sherbrooke"
tokens = tokenizer.tokenize(prompt)
print(tokens)

['▁Je', '▁suis', '▁au', '▁campus', '▁de', '▁l', "'", 'Université', '▁de', '▁Sherbrooke']


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

['Je', 'Ġsu', 'is', 'Ġau', 'Ġcampus', 'Ġde', 'Ġl', "'", 'Un', 'ivers', 'itÃ©', 'Ġde', 'ĠSher', 'bro', 'oke']


Comparez avec le _tokenizer_ de _BART_ :

In [ ]:
tokenizer_BART = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")
tokens_BART = tokenizer_BART.tokenize(prompt)
print(tokens_BART)

['Je', 'Ġsu', 'is', 'Ġau', 'Ġcampus', 'Ġde', 'Ġl', "'", 'Un', 'ivers', 'itÃ©', 'Ġde', 'ĠSher', 'bro', 'oke']


# 3️⃣ Test du modèle préentraîné

Les _token_ générés ci-dessus ne sont pas formattés correctement pour le modèle _PyTorch_.

Il faut que le _tokenizer_ nous retourne un _tensor PyTorch_ contenant des nombres représentant nos _token_.

Cette représentation est donnée au modèle qui les transforment ensuite en vecteur de nombre via son bloc d'_input embedding_. Au final, le modèle nous retourne également un _tensor PyTorch_. Celui-ci doit être décodé pour obtenir le texte résultant.

Notez que la sortie du modèle n'est pas très utile. Nous ne l'avons pas encore _fine-tune_ avec une tâche.

In [ ]:
tokens = tokenizer("Je suis au campus de l'Université de Sherbrooke", return_tensors="pt")
output = model.generate(**tokens)
print(output)

decoded_output = tokenizer.decode(output[0], skip_special_tokens=True)
print(decoded_output)

tensor([[    2,     0,   382,   849,    63, 17701,    10,     6, 49928,  4945,
            10,  3573, 49922,     2]])
Je suis au campus de l'Université de Montréal.


# 4️⃣ Acquisition du jeux de données utilisé pour l'entraînement du modèle

**Section à venir**

# 5️⃣ Plateforme _Weights & Biases_
_W&B_ est un outil commun pour le suivi du développement de modèles. Il permet notamment une API simple pour entraîner des modèles et envoyer les statistiques d'entraînement à un projet _W&B_. Ces stats peuvent être visualisées et partagées au sein d'une équipe.

1.   Créez vous un compte chez https://wandb.ai/
2.   Logger vous dans votre compte _W&B_ dans la session qui exécutera votre code d'entraînement. Dans _Colab_, vous pouvez appeler du _bash_ directement dans une cellule de code. Préfacez votre code _bash_ avec `!`.
3. Entrez votre code API. Trouvez le ici : https://wandb.ai/authorize
4.   Importez le module _Python `wandb`. Vous pouvez aussi vous logger dans _W&B_ avec l'API en appelant `wandb.login()`.
4.   Utilisez l'API et observer les résultats d'entraînements sur le site.

**Essayez la démo ici : https://colab.research.google.com/github/wandb/examples/blob/master/colabs/intro/Intro_to_Weights_%26_Biases.ipynb**

In [ ]:
!wandb login

wandb: Currently logged in as: rosskay (INF717) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin
